# Quantiza Model with Post-training Optimization Tools

### Prepare Environment
Before you start with Apis delivered by bigdl-nano, you have to make sure BigDL-Nano is correctly installed for PyTorch. If not, please follow [this](../../../../../docs/readthedocs/source/doc/Nano/Overview/nano.md) to set up your environment.<br><br>
The POT(Post-training Optimization Tools) is provided by OpenVINO toolkit. To use POT, you need to install OpenVINO.
```bash
pip install openvino-dev
```

### Step 0: Load Data
We used the [Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/) for demo, which contains 37 categories with roughly 200 images for each classes.

In [18]:
from torchvision.datasets import OxfordIIITPet
from torch.utils.data.dataloader import DataLoader
from torchvision import transforms
from torch.utils.data.dataloader import DataLoader
data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=.5, hue=.3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
data_set = OxfordIIITPet(root="./data/", transform=data_transforms,
                         target_transform=transforms.Lambda(lambda label: torch.tensor(label, dtype=torch.long)))
data_loader = DataLoader(data_set, batch_size=32, shuffle=True)

### Step 1: Custom Model
Regarding the model, we used pretrained torchvision.models.resnet18. More details, please refer to [here](https://pytorch.org/vision/0.12/generated/torchvision.models.resnet18.html?highlight=resnet18)

In [19]:
from torchvision.models import resnet18
import torch
import torch.nn as nn
# define your own model
model_ft = resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, len(data_set.classes))
loss_ft = nn.CrossEntropyLoss()
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
from bigdl.nano.pytorch import Trainer
model = Trainer.compile(model_ft, loss_ft, optimizer_ft)
# (Optional) Something else, like training ...
trainer = Trainer(max_epochs=5)
trainer.fit(model, train_dataloader=data_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 11.2 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.782    Total estimated model params size (MB)


Epoch 4: 100%|██████████| 115/115 [00:56<00:00,  2.06it/s, loss=0.393, v_num=42] 


### Step 2: Quantization with Post-training Optimization Tools
Add quantization as follow, accelerator='openvino' means using OpenVINO POT to do quantization.

In [20]:
from torchmetrics import Accuracy
ov_q_model = trainer.quantize(model, accelerator="openvino", calib_dataloader=data_loader)
batch = torch.stack([data_set[0][0], data_set[1][0]])
ov_q_model(batch)

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/tmp/tmp_h5tiarj/tmp.onnx
	- Path for generated IR: 	/tmp/tmpui3v8zys
	- IR output name: 	tmp
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Source layout: 	Not specified
	- Target layout: 	Not specified
	- Layout: 	Not specified
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- User transformations: 	Not specified
	- Reverse input channels: 	False
	- Enable IR generation for fixed input shape: 	False
	- Use the transformations config file: 	None
Advanced parameters:
	- Force the usage of legacy Frontend of Model Optimizer for model conversion into IR: 	False
	- Force the usage of new Frontend of Model Optimizer for 

tensor([[ 1.7329e+01, -3.2855e+00, -1.1449e+00, -9.7942e-01, -1.5179e-01,
          9.2083e+00, -3.6007e+00, -5.1113e-01, -1.1325e+00,  3.0623e+00,
          3.0486e+00,  5.7270e+00, -1.7772e+00, -4.8688e+00,  4.2304e-01,
         -6.5337e+00, -3.7628e+00, -2.1463e+00, -5.7218e+00, -3.9071e+00,
          4.8865e+00,  1.8713e+00, -6.4810e+00, -1.7373e+00,  9.8768e-01,
         -4.9941e+00,  2.6608e+00,  4.8284e+00, -2.7764e+00, -4.6965e+00,
         -3.5127e+00,  5.5518e+00,  3.3330e+00,  2.4686e+00, -5.0390e+00,
         -1.3475e+00, -2.0684e-01],
        [ 1.1245e+01, -3.0037e+00, -1.7740e-04, -1.8865e+00,  1.3851e+00,
          9.7956e+00, -2.2753e+00,  5.0934e-01, -4.8474e-01,  1.0725e+00,
          4.4877e+00,  4.0109e+00, -1.5552e+00, -4.1258e+00, -1.4994e+00,
         -5.8474e+00, -2.3569e+00, -7.3282e-01, -5.3210e+00, -3.4000e+00,
          2.7175e+00,  2.7583e+00, -4.3755e+00, -1.7895e+00,  8.6411e-01,
         -2.5912e+00,  1.1335e+00,  2.4955e+00, -1.2317e+00, -5.0416e+00,
  